# Exercise 0.1
## Introduction to EM data, neuron segmentation and gunpowder
### A simple gunpowder example pipeline for loading and manipulating data

Become familiar with gunpowder. Read and try to understand the following code as well as the general principle behind gunpowder. Read the introductory example in the gunpowder documentation http://funkey.science/gunpowder and gunpowder tutorial http://funkey.science/gunpowder/tutorial.html

In [24]:
from __future__ import print_function
from gunpowder import *
from gunpowder.tensorflow import *
import numpy as np
import os

# A simple gunpowder pipeline for loading and manipulating data:
data_dir = '../../jan/segmentation/data'

# Define gunpowder variables of interest:
raw = ArrayKey('RAW') # Raw EM data
labels = ArrayKey('GT_LABELS') # ground truth neuron segmentation 
affinities = ArrayKey('GT_AFFINITIES') # affinities

# Voxel size is the physical size of one voxel (=3D pixel) in nm.
voxel_size = Coordinate((40, 4, 4))
batch_size = Coordinate((50,1000,1000)) * voxel_size

# Request all the data you need for training:
request = BatchRequest()
request.add(raw, batch_size)
request.add(labels, batch_size)
request.add(affinities, batch_size)

selec_roi = Roi(offset=(3000,5000,5100), shape=batch_size)
request[raw].roi = selec_roi
request[labels].roi = selec_roi
request[affinities].roi = selec_roi

# Request a snapshot s.t. you are able to visualize the data in your pipeline.
snapshot_request = BatchRequest({
    raw: request[raw],
    labels: request[labels],
    affinities: request[affinities]
})

# Note that the data only provides raw and neuron_ids which is the neuron segmentation.
# However, we need affinities which we can generate from neuron_ids by using gunpowder (see below):
data_sources = tuple(
    N5Source(
        os.path.join(data_dir, sample + '.n5'),
        datasets = {
            raw: 'volumes/raw',
            labels: 'volumes/labels/neuron_ids',
        },
        array_specs = {
            raw: ArraySpec(interpolatable=True),
            labels: ArraySpec(interpolatable=False)
        }
    ) +
    Normalize(raw)
    for sample in ["sample_C"]
    )

# Define a neighborhood for affinities:
neighborhood = [[-1, 0, 0], [0, -1, 0], [0, 0, -1]]

In [5]:
import math
# Build the actual data-pipeline:

def basic_pipeline():
    pipeline = (
        data_sources +
        RandomProvider() +
        AddAffinities(neighborhood,
                     labels=labels,
                     affinities=affinities) +
        Snapshot(dataset_names={raw: 'volumes/raw',
                  labels: 'volumes/labels/neuron_ids',
                  affinities: 'volumes/affinities'},
                  output_filename="snapshot_basic.hdf"))
    return pipeline

def simple_augment_pipeline():
    pipeline = (
        data_sources +
        RandomProvider() +
        SimpleAugment(transpose_only=[1, 2]) +
        AddAffinities(neighborhood,
                     labels=labels,
                     affinities=affinities) +
        Snapshot(dataset_names={raw: 'volumes/raw',
                  labels: 'volumes/labels/neuron_ids',
                  affinities: 'volumes/affinities'},
                  output_filename="snapshot_simple.hdf"))
    return pipeline

def intensity_augment_pipeline():
    pipeline = (
        data_sources +
        RandomProvider() +
        IntensityAugment(raw, 0.9, 1.1, -0.1, 0.1, z_section_wise=True) +    
        AddAffinities(neighborhood,
                     labels=labels,
                     affinities=affinities) +
        Snapshot(dataset_names={raw: 'volumes/raw',
                  labels: 'volumes/labels/neuron_ids',
                  affinities: 'volumes/affinities'},
                  output_filename="snapshot_intensity.hdf"))
    return pipeline

def elastic_augment_pipeline():
    pipeline = (
        data_sources +
        RandomProvider() +
        ElasticAugment(
            control_point_spacing=[4,40,40],
            jitter_sigma=[0,2,2],
            rotation_interval=[0,math.pi/2.0],
            prob_slip=0.05,
            prob_shift=0.05,
            max_misalign=10,
            subsample=8) +
        AddAffinities(neighborhood,
                     labels=labels,
                     affinities=affinities) +
        Snapshot(dataset_names={raw: 'volumes/raw',
                  labels: 'volumes/labels/neuron_ids',
                  affinities: 'volumes/affinities'},
                  output_filename="snapshot_elastic.hdf"))
    return pipeline


def king_of_the_north():
    pipeline = (
        data_sources +
        RandomProvider() +
        ElasticAugment(
            control_point_spacing=[4,40,40],
            jitter_sigma=[0,2,2],
            rotation_interval=[0,math.pi/2.0],
            prob_slip=0.05,
            prob_shift=0.05,
            max_misalign=10,
            subsample=8) +
        SimpleAugment(transpose_only=[1, 2]) +
        IntensityAugment(raw, 0.9, 1.1, -0.1, 0.1, z_section_wise=True) +
        AddAffinities(neighborhood,
                     labels=labels,
                     affinities=affinities) +
        Snapshot(dataset_names={raw: 'volumes/raw',
                  labels: 'volumes/labels/neuron_ids',
                  affinities: 'volumes/affinities'},
                  output_filename="snapshot_king_of_the_north.hdf"))
    
    return pipeline

pipeline_setups={"basic": basic_pipeline,
                 "simple": simple_augment_pipeline,
                 "intensity": intensity_augment_pipeline,
                 "elastic": elastic_augment_pipeline}

try:
    pipeline_setups.update({"king_of_the_north": king_of_the_north})
except:
    pass

# Exercise 0.2
## Choose a pipeline and understand how different augmentation types influence the output.

### You can choose between:
1. basic (No augmentations)
2. simple
3. intensity 
4. elastic
5. (*If you implemented it: king_of_the_north)

Try to find out what each of these are actually doing!

In [26]:
# Choose a pipeline, run it and observe the differences in neuroglancer.
pipeline_name = "king_of_the_north"

if not os.path.exists("./snapshots"):
    os.makedirs("./snapshots")
pipeline_setup = pipeline_setups[pipeline_name]

with build(pipeline_setup()) as b:
    b.request_batch(request)

## Visualize data using neuroglancer

The prior cell generated a file in your working directory containg the raw EM data, neuron segmentation and affinities. The following script lets you view the data you just generated and will change depending on which augmentation pipeline you choose. Run both cells below and click on the resulting link. You can enable and disable layers by clicking on them. Explore the data. What do the different colors of the affinities mean? How does it relate to the segmentation? How is augmentation affecting the data?

If you double click on a particular segment neuroglancer allows you to view the 3D mesh of the object.

### Dataset:
The data you are looking at are a tiny subset of the **whole** adult Drosophila brain, imaged via serial section transmission electron microscopy (SSTEM) ([Zheng et al. 2018](https://www.sciencedirect.com/science/article/pii/S0092867418307876)). If you are curious and want to get a sense of the scale, [here](https://fafb.catmaid.virtualflybrain.org/?pid=1&zp=127040&yp=253133.36932477387&xp=585045.0300033365&tool=navigator&sid0=1&s0=7.400000000000001) you can browse the entire dataset. Zooming in and out is a lot of fun, don't get lost.

The particular cutout you are looking at here, is itself a small cutout of the [CREMI](https://cremi.org/) challenge volumes: a neuron and synapse segmentation challenge that provides manually acquired ground truth and provides a way to quantitatively compare competing algorithmic approaches.

##### REPLACE THE IP BELOW WITH YOURS:

In [27]:
import neuroglancer
import funlib.show.neuroglancer as funlibng
import daisy
import numpy as np

# !!! IMPORTANT: Replace my_ip with your public paperspace ip as shown in your browser !!!

def view_snapshot(pipeline_name, my_ip="74.82.31.73", snapshot_file="./snapshots/snapshot"):
    neuroglancer.set_server_bind_address('0.0.0.0', 8889)
    snapshot_file = snapshot_file + "_" + pipeline_name + ".hdf"
    
    affs_ds = daisy.open_ds(snapshot_file, 'volumes/affinities')
    affs_ds.data = np.array(affs_ds.data, dtype=np.float32)
    labels_ds = daisy.open_ds(snapshot_file, 'volumes/labels/neuron_ids')
    raw_ds = daisy.open_ds(snapshot_file, 'volumes/raw')

    neuroglancer.set_server_bind_address('0.0.0.0')
    viewer = neuroglancer.Viewer()
    with viewer.txn() as s:
        funlibng.add_layer(s, raw_ds, 'raw')
        funlibng.add_layer(s, labels_ds, 'labels')
        funlibng.add_layer(s, affs_ds, 'affinities', shader='rgb')

    print(viewer.__str__().replace("localhost", my_ip))

In [28]:
view_snapshot(pipeline_name)

http://74.82.31.73:36615/v/a31d6e977d47b918bb5a6157e0accfd3ce39800b/


# Exercise 1
## Using gunpowder with tensorflow for training a 3D-UNet for affinity prediction
#### mknet.py

In this exercise we ask you to pick a network of your choosing to create a computation graph for your neural network. Follow the instructions in the code and make use of help and documentations.

As usual if you are not sure about what a function does or what its argument is navigate to the function and press ```shift + tab``` to display arguments and documentation.

In [6]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6659018488027667204
]


If the output above doesn't mention the word GPU scream HELP!

In [30]:
from funlib.learn.tensorflow import models
import malis
import tensorflow as tf
import json


def create_network(name, input_shape, num_fmaps, fmap_inc_factors, downsample_factors):

    tf.reset_default_graph()

    with tf.variable_scope('setup0'):

        raw = tf.placeholder(tf.float32, shape=input_shape)
        raw_batched = tf.reshape(raw, (1, 1) + input_shape)

        unet, _, _ = models.unet(
                raw_batched,
                num_fmaps,
                fmap_inc_factors,
                downsample_factors)

        affs_batched, _ = models.conv_pass(
            unet,
            kernel_sizes=[1],
            num_fmaps=3,
            activation='sigmoid',
            name='affs')

        output_shape_batched = affs_batched.get_shape().as_list()
        output_shape = output_shape_batched[1:] # strip the batch dimension

        affs = tf.reshape(affs_batched, output_shape)

        gt_affs = tf.placeholder(tf.float32, shape=output_shape)
        affs_loss_weights = tf.placeholder(tf.float32, shape=output_shape)

        loss = tf.losses.mean_squared_error(
            gt_affs,
            affs,
            affs_loss_weights)

        summary = tf.summary.scalar('setup0', loss)

        opt = tf.train.AdamOptimizer(
            learning_rate=0.5e-4,
            beta1=0.95,
            beta2=0.999,
            epsilon=1e-8)
        optimizer = opt.minimize(loss)

        output_shape = output_shape[1:]
        print("input shape : %s"%(input_shape,))
        print("output shape: %s"%(output_shape,))

        tf.train.export_meta_graph(filename=name + '.meta')

        config = {
            'raw': raw.name,
            'affs': affs.name,
            'gt_affs': gt_affs.name,
            'affs_loss_weights': affs_loss_weights.name,
            'loss': loss.name,
            'optimizer': optimizer.name,
            'input_shape': input_shape,
            'output_shape': output_shape,
            'summary': summary.name
        }

        config['outputs'] = {'affs': {"out_dims": 3, "out_dtype": "uint8"}}

        with open(name + '.json', 'w') as f:
            json.dump(config, f)
        
        if name=="arya":
            print("\nVALAR MORGHULIS")
        if name=="jon":
            print("\nFOR THE NORTH")
        if name=="daenerys":
            print("\nDRACARYS")
        if name=="cersei":
            print("\nJAIME!")

Below you find a selection of your favorite 3D-UNets in Westeros. Make use of the following documentation to decide which of these networks is to your liking.

In [27]:
models.unet?

In [41]:
arya_stark = {"name": "arya", 
        "input_shape": (84, 268, 268),
        "num_fmaps": 24,
        "fmap_inc_factors": 2,
        "downsample_factors": [[1,3,3],[4,12,12]]}

jon_snow = {"name": "jon", 
        "input_shape": (84, 268, 268),
        "num_fmaps": 12,
        "fmap_inc_factors": 5,
        "downsample_factors": [[1,3,3],[1,3,3],[3,3,3]]}

daenerys_stormborn = {"name": "daenerys", 
                      "input_shape": (84, 268, 268),
                      "num_fmaps": 12,
                      "fmap_inc_factors": 1,
                      "downsample_factors": [[1,3,3],[1,3,3],[3,3,3]]}

cersei_lennister = {"name": "cersei", 
                    "input_shape": (84, 268, 268),
                    "num_fmaps": 12,
                    "fmap_inc_factors": 1,
                    "downsample_factors": [[1,3,3],[1,3,3],[3,3,3]]}

networks = {"arya": arya_stark, "jon": jon_snow, "daenerys": daenerys_stormborn, "cersei": cersei_lennister}

### Pick your network
Choose between Arya, Jon, Daenerys and Cersei described above.

In [42]:
network = "arya"
create_network(**networks[network])

Creating U-Net layer 0
f_in: (1, 1, 84, 268, 268)
number of variables added: 16248, new total: 16248
    Creating U-Net layer 1
    f_in: (1, 24, 80, 88, 88)
    number of variables added: 93408, new total: 109656
        Creating U-Net layer 2
        f_in: (1, 48, 19, 7, 7)
        bottom layer
        f_out: (1, 96, 15, 3, 3)
        number of variables added: 373440, new total: 483096
    g_out: (1, 96, 15, 3, 3)
    g_out_upsampled: (1, 48, 60, 36, 36)
    f_left_cropped: (1, 48, 60, 36, 36)
    f_right: (1, 96, 60, 36, 36)
    f_out: (1, 48, 56, 32, 32)
    number of variables added: 2840976, new total: 3324072
g_out: (1, 48, 56, 32, 32)
g_out_upsampled: (1, 24, 56, 96, 96)
f_left_cropped: (1, 24, 56, 96, 96)
f_right: (1, 48, 56, 96, 96)
f_out: (1, 24, 52, 92, 92)
number of variables added: 57096, new total: 3381168
input shape : (84, 268, 268)
output shape: [52, 92, 92]

VALAR MORGHULIS


# Exercise 2
## A gunpowder training pipeline

In this exercise we ask you to use gunpowder in combination with tensorflow to build a training pipeline for affinity prediction. Fill in the gaps in the template code below and train your network!

In [3]:
from __future__ import print_function
import sys
from gunpowder import *
from gunpowder.tensorflow import *
import os
import math
import json
import tensorflow as tf
import numpy as np
import logging

logging.basicConfig(level=logging.ERROR)

data_dir = '../../jan/segmentation/data'

samples = [
    'sample_A',
    'sample_B',
    'sample_C'
]

neighborhood = [[-1, 0, 0], [0, -1, 0], [0, 0, -1]]

def train_until(max_iteration):

    if tf.train.latest_checkpoint('.'):
        trained_until = int(tf.train.latest_checkpoint('.').split('_')[-1])
    else:
        trained_until = 0
    if trained_until >= max_iteration:
        return

    with open('train_net.json', 'r') as f:
        config = json.load(f)

    raw = ArrayKey('RAW')
    labels = ArrayKey('GT_LABELS')
    labels_mask = ArrayKey('GT_LABELS_MASK')
    affs = ArrayKey('PREDICTED_AFFS')
    gt = ArrayKey('GT_AFFINITIES')
    gt_mask = ArrayKey('GT_AFFINITIES_MASK')
    gt_scale = ArrayKey('GT_AFFINITIES_SCALE')
    affs_gradient = ArrayKey('AFFS_GRADIENT')

    voxel_size = Coordinate((40, 4, 4))
    input_size = Coordinate(config['input_shape'])*voxel_size
    output_size = Coordinate(config['output_shape'])*voxel_size
    context = output_size/2
    print('CONTEXT: ', context)

    request = BatchRequest()
    request.add(raw, input_size)
    request.add(labels, output_size)
    request.add(labels_mask, output_size)
    request.add(gt, output_size)
    request.add(gt_mask, output_size)
    request.add(gt_scale, output_size)

    snapshot_request = BatchRequest({
        affs: request[gt],
        affs_gradient: request[gt]
    })

    data_sources = tuple(
        N5Source(
            os.path.join(data_dir, sample + '.n5'),
            datasets = {
                raw: 'volumes/raw',
                labels: 'volumes/labels/neuron_ids',
                labels_mask: 'volumes/labels/mask',
            },
            array_specs = {
                raw: ArraySpec(interpolatable=True),
                labels: ArraySpec(interpolatable=False),
                labels_mask: ArraySpec(interpolatable=False)
            }
        ) +
        Normalize(raw) +
        Pad(labels, context) +
        Pad(labels_mask, context) +
        RandomLocation() +
        Reject(mask=labels_mask)
        for sample in samples
    )


    train_pipeline = (
        data_sources +
        RandomProvider() +
        ElasticAugment(
            control_point_spacing=[4,40,40],
            jitter_sigma=[0,2,2],
            rotation_interval=[0,math.pi/2.0],
            prob_slip=0.05,
            prob_shift=0.05,
            max_misalign=10,
            subsample=8) +
        SimpleAugment(transpose_only=[1, 2]) +
        IntensityAugment(raw, 0.9, 1.1, -0.1, 0.1, z_section_wise=True) +
        GrowBoundary(labels, labels_mask, steps=1, only_xy=True) +
        AddAffinities(
            neighborhood,
            labels=labels,
            affinities=gt,
            labels_mask=labels_mask,
            affinities_mask=gt_mask) +
        BalanceLabels(
            gt,
            gt_scale,
            gt_mask) +
        DefectAugment(
            raw,
            prob_missing=0.03,
            prob_low_contrast=0.01,
            contrast_scale=0.5,
            axis=0) +
        IntensityScaleShift(raw, 2,-1) +
        PreCache(cache_size=40,
                 num_workers=10) +
        Train(
            'train_net',
            optimizer=config['optimizer'],
            loss=config['loss'],
            inputs={
                config['raw']: raw,
                config['gt_affs']: gt,
                config['affs_loss_weights']: gt_scale,
            },
            outputs={
                config['affs']: affs
            },
            gradients={
                config['affs']: affs_gradient
            },
            summary=config['summary'],
            log_dir='log',
            save_every=10000) +
        IntensityScaleShift(raw, 0.5, 0.5) +
        Snapshot({
                raw: 'volumes/raw',
                labels: 'volumes/labels/neuron_ids',
                gt: 'volumes/gt_affinities',
                affs: 'volumes/pred_affinities',
                gt_mask: 'volumes/labels/gt_mask',
                labels_mask: 'volumes/labels/mask',
                affs_gradient: 'volumes/affs_gradient'
            },
            dataset_dtypes={
                labels: np.uint64
            },
            every=1000,
            output_filename='batch_{iteration}.hdf',
            additional_request=snapshot_request) +
        PrintProfilingStats(every=10)
    )

    print("Starting training...")
    with build(train_pipeline) as b:
        for i in range(max_iteration - trained_until):
            b.request_batch(request)
    print("Training finished")

if __name__ == "__main__":

    iteration = 500000
    train_until(iteration)

INFO:gunpowder.tensorflow.local_server:Creating local tensorflow server
INFO:gunpowder.tensorflow.local_server:Server running at b'grpc://localhost:36565'
INFO:gunpowder.tensorflow.nodes.train:Initializing tf session, connecting to b'grpc://localhost:36565'...
INFO:gunpowder.tensorflow.nodes.train:Reading meta-graph...


CONTEXT:  (960, 112, 112)
Starting training...


INFO:gunpowder.tensorflow.nodes.train:No checkpoint found
INFO:gunpowder.nodes.precache:starting new set of workers...
INFO:gunpowder.producer_pool:terminating workers...
INFO:gunpowder.producer_pool:joining workers...
INFO:gunpowder.producer_pool:done


InvalidArgumentError: Conv3DBackpropInputOpV2 only supports NDHWC on the CPU.
	 [[Node: setup0/decoder_0_layer_2/unet_up_3_to_2/conv3d_transpose = Conv3DBackpropInputV2[T=DT_FLOAT, Tshape=DT_INT32, data_format="NCDHW", dilations=[1, 1, 1, 1, 1], padding="VALID", strides=[1, 1, 3, 3, 3], _device="/job:local/replica:0/task:0/device:CPU:0"](setup0/gradients/setup0/decoder_0_layer_2/concat_grad/Shape, setup0/decoder_0_layer_2/unet_up_3_to_2/kernel/read, setup0/unet_layer_3_left_1/Relu)]]

Caused by op 'setup0/decoder_0_layer_2/unet_up_3_to_2/conv3d_transpose', defined at:
  File "/home/paperspace/anaconda3/envs/segmentation/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/paperspace/anaconda3/envs/segmentation/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/paperspace/.local/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/paperspace/

# Exercise 3

## Monitor the training progress

Visualize the generated training snapshots with neuroglancer and observe loss curves using tensorboard.


#### Snapshots

In [ ]:
import neuroglancer
import funlib.show.neuroglancer as funlibng
import daisy
import numpy as np
"""
data_file = ...
"""

gt_affs_ds = daisy.open_ds(data_file, 'volumes/gt_affinities')
gt_affs_ds.data = np.array(gt_affs_ds.data, dtype=np.float32)

pred_affs_ds = daisy.open_ds(data_file, 'volumes/pred_affinities')
pred_affs_ds.data = np.array(pred_affs_ds.data, dtype=np.float32)

gradients_ds = daisy.open_ds(data_file, 'volumes/affs_gradient')

labels_ds = daisy.open_ds(data_file, 'volumes/labels/neuron_ids')
raw_ds = daisy.open_ds(data_file, 'volumes/raw')

# Replace my_ip with your public paperspace ip as shown in your browser
my_ip = "172.83.14.204"
neuroglancer.set_server_bind_address('0.0.0.0', 8889)

viewer = neuroglancer.Viewer()
with viewer.txn() as s:
    funlibng.add_layer(s, raw_ds, 'raw')
    funlibng.add_layer(s, labels_ds, 'labels')
    funlibng.add_layer(s, gt_affs_ds, 'gt_affinities', shader='rgb')
    funlibng.add_layer(s, pred_affs_ds, 'pred_affinities', shader='rgb')
    funlibng.add_layer(s, gradients_ds, 'gradients')

print(viewer.__str__().replace("localhost", my_ip))

#### Loss
To visualize your loss using tensorboard open a terminal and execute: 
```tensorboard --logdir=path/to/log-directory```